# STEP3 TA IndéMode  データ取得 不動産

>### 環境構築

ターミナルにて  
 ● pip install requests  
 ● pip install beautifulsoup4  
 ● pip install gspread  
 ● pip install google-auth

>### 活用サイト

テナントショップ  


>### コード

In [14]:
# ライブラリインポート
from time import sleep
import requests
from bs4 import BeautifulSoup
import pandas as pd
import gspread 
from google.oauth2 import service_account
import json
import sqlite3

# スプレッドシートの認証設定
#SP_CREDENTIAL_FILE = '  ' #個人の認証キーのjsonファイルのパスを記入
#SP_COPE = [
#    'https://www.googleapis.com/auth/drive',
#    'https://spreadsheets.google.com/feeds'
#]
#credentials = service_account.Credentials.from_service_account_file(SP_CREDENTIAL_FILE, scopes=SP_COPE)
#gc = gspread.authorize(credentials)

#スプレッドシート/シート指定
#SP_SHEET_KEY = '1474-fT6RkuGOvPwjyCJIEtD6qrNjI6EP2bw3vtHvLBc' #スプレッドシート
#SP_SHEET = '空き店舗DB'

# 情報格納のための空配列を用意
TenantNo = [] # Number
TenantName = [] #name
TenantPrice = [] # 家賃
TenantPrice_per_unit_area = [] # 坪単価
TenantStation = [] # 駅
TenantLine = [] # 路線
TenantWalk = [] # 駅徒歩
Tenantlocate = [] # 住所
Tenantbuild = [] # 築年数
TenantFloor = [] # 階数
TenantSize = [] # 平米
TenantURL = [] # URL

# HTML取得
URL = 'https://www.tenant-shop.com/index.php?ac=2&c=12&a[]=317&pa=14&f6=1&sp=3&mv=2&p={}'

# 複数ページ取得のためのループ処理
for i in range(1,6):

    # 変数trget_urlに、アクセス先のURLを格納する
    load_url = URL.format(i)

    #サーバー負荷低減のための1s待機
    sleep(2)

    #print()してload_url確認
    print(load_url)

    # リクエスト
    html = requests.get(load_url)

    # リクエスト結果を確認
    #result = html.text
    #print(result)

    # 文字コード変換 (→ utf-8 )
    html.encoding = 'utf-8'

    # 文字化けしていないかhtmlを確認
    #print(html.text)

    # BeautifulSoup(解析したいデータ,解析する方法)を指定し、soupに代入
    soup = BeautifulSoup(html.content, "html.parser")
    #print(soup)

    # sectionデータ(1物件分データ)の取得
    elemsO = soup.find_all("tr",class_="odd result-row")
    elemsE = soup.find_all("tr",class_="even result-row")
    elems = elemsO + elemsE
    #df = pd.DataFrame(elems)

    # 取得したセクションすべての数を確認します。
    #print("タグの数:", len(elems))
    #print(df)
    
    # すべてのliタグを検索して、その文字列を表示する
    #for elem in elems:
    #	price = elems.find_all("div",class_="price")
    #	print(price.text)

    # elemsからsectionを1つずつ取り出してelemに代入。
    for elem in elems:
        if (elem.select_one('.pubestno') != None):
            elem1 = elem.select_one(".pubestno").text #テナント名
        else:
            elem1 = "-1"

        if (elem.select_one('.price') != None):
            elem2 = elem.select_one('.price').text.replace("万","").replace("(税込)","").replace("(税別)","").replace(" ","").replace("\n","")# 値段をhs2に代入
        else:
            elem2 = -1 # 値段が記入されていない場合があるので、わかりやすく-1にしておきましょう。
        
        elem3 = elem.select_one('.info')
        elem40 = elem3.select("div")[1].text
        #elem40 = re.split('[ |　|\xa0]', elem40)
        #elem41 = elem40[0]
        #elem42 = elem40[1]
        #if (elem40[2] != None):
        #    elem43 =elem40[2]
        #else:
        #      elem43 = -1
        elem5 = elem3.select("div")[2].text
        elem6 = elem.select_one(".floor").text.replace("㎡","")
        elem7 = elem.select(".add")[-1].text
        elem8 = elem3.select("div")[0].text

        if (elem.select_one('.smallText') != None):
            elem9 = elem.select_one('.smallText').text.replace("万/坪","").replace("(","").replace(")","")
        else:
            elem9 = -1 # 値段が記入されていない場合があるので、わかりやすく-1にしておきましょう。

        # それぞれ抽出したデータを配列に追加
        #TenantNo.append(elem1) 
        TenantPrice.append(elem2) 
        TenantPrice_per_unit_area.append(elem9) 
        #TenantLine.append(elem41)
        TenantStation.append(elem40)
        #TenantWalk.append(elem43)
        Tenantbuild.append(elem5)
        TenantSize.append(elem6)
        TenantFloor.append(elem7)
        Tenantlocate.append(elem8) 
        
    # 物件数確認
    print('検索物件数', len(Tenantlocate))

# データ定義
data_list = {
    #"物件No" : TenantNo,
    "住所" : Tenantlocate,
    "面積(㎡)" : TenantSize,
    "賃料(万円)" : TenantPrice,
    "坪単価(万円/坪)" : TenantPrice_per_unit_area,    
    #"Line" : TenantLine,
    "駅" : TenantStation,
    "築年" : Tenantbuild,
    "階" : TenantFloor,
}

# データフレーム作成
df = pd.DataFrame(data_list)

# 重複データ処理
df.drop_duplicates(inplace=True)
df.reset_index(drop=True,inplace=True)
# display(df)

# csv保存
#df.to_csv('貸店舗・テナント物件一覧_テナントショップ.csv', index=False)

# スプレッドシート/シートを選択して初期化既存データ削除）
#sh = gc.open_by_key(SP_SHEET_KEY)
#worksheet = sh.worksheet(SP_SHEET)
#worksheet.clear() 

# DataFrameをシートに書き込む
#worksheet.update([filtered_df.columns.values.tolist()] + filtered_df.values.tolist())

#print('スプレッドシートへの書き込みが完了しました')

# SQLでDBにデータ保存
# SQLiteデータベースへの接続
db_path = "STEP3チーム課題_TA_IndéMode_DB.db"
conn = sqlite3.connect(db_path) 

# データフレームをSQLiteデータベースに書き込む
table_name = '店舗一覧'  # テーブル名を適切なものに変更してください
df.to_sql(table_name, conn, if_exists='replace', index=False)

print(db_path + ' への書き込みが完了しました')

https://www.tenant-shop.com/index.php?ac=2&c=12&a[]=317&pa=14&f6=1&sp=3&mv=2&p=1
検索物件数 20
https://www.tenant-shop.com/index.php?ac=2&c=12&a[]=317&pa=14&f6=1&sp=3&mv=2&p=2
検索物件数 40
https://www.tenant-shop.com/index.php?ac=2&c=12&a[]=317&pa=14&f6=1&sp=3&mv=2&p=3
検索物件数 49
https://www.tenant-shop.com/index.php?ac=2&c=12&a[]=317&pa=14&f6=1&sp=3&mv=2&p=4
検索物件数 49
https://www.tenant-shop.com/index.php?ac=2&c=12&a[]=317&pa=14&f6=1&sp=3&mv=2&p=5
検索物件数 49
STEP3チーム課題_TA_IndéMode_DB.db への書き込みが完了しました


In [15]:
df.head()

,住所,面積(㎡),賃料(万円),坪単価(万円/坪),駅,築年,階
0,千代田区岩本町1-8-3,77.68,36.19,1.54,東京メトロ⽇⽐⾕線「⼩伝⾺町」 2,1997年12月(平成9年12月),階:2F／9F
1,千代田区平河町1-2-2,72.56,33.80,1.54,東京メトロ半蔵⾨線「半蔵⾨」 5,1985年7月(昭和60年7月),階:4F／7F
2,千代田区飯田橋1-9-5,36.62,35.00,3.16,東京メトロ東西線 飯田橋駅 2,1980年(昭和55年),階:1F／5F
3,千代田区麹町3-12-5,102.48,71.61,2.31,東京メトロ有楽町線「麹町」 2,1992年2月(平成4年2月),階:4F／9F
4,千代田区外神田1,90.24,81.87,3.00,秋葉原 1,2014年6月(平成26年6月),階:9F／10F
